# Tutorial: Upload your own Catalog

The purpose of this tutorial is to illustrate how you can construct your own catalog to upload to the FastSpecFit web-app at https://fastspecfit.desi.lbl.gov.

John Moustakas  
Siena College  
February 2023

In [1]:
import os
import fitsio
import numpy as np
from astropy.table import Table

#### Read the data

In [2]:
datadir = '/global/cfs/cdirs/desi/spectro/fastspecfit/fuji/v2.0/catalogs'
%time fast = Table(fitsio.read(os.path.join(datadir, 'fastspec-fuji.fits'), 'FASTSPEC'))
%time meta = Table(fitsio.read(os.path.join(datadir, 'fastspec-fuji.fits'), 'METADATA'))

CPU times: user 31.3 s, sys: 8.93 s, total: 40.2 s
Wall time: 51.7 s
CPU times: user 2.77 s, sys: 1.09 s, total: 3.86 s
Wall time: 4.08 s


In [3]:
print(meta.colnames)

['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'TILEID_LIST', 'RA', 'DEC', 'COADD_FIBERSTATUS', 'CMX_TARGET', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SCND_TARGET', 'SV1_DESI_TARGET', 'SV1_BGS_TARGET', 'SV1_MWS_TARGET', 'SV2_DESI_TARGET', 'SV2_BGS_TARGET', 'SV2_MWS_TARGET', 'SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET', 'Z', 'ZWARN', 'DELTACHI2', 'SPECTYPE', 'Z_RR', 'TSNR2_BGS', 'TSNR2_LRG', 'TSNR2_ELG', 'TSNR2_QSO', 'TSNR2_LYA', 'PHOTSYS', 'LS_ID', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'FIBERTOTFLUX_G', 'FIBERTOTFLUX_R', 'FIBERTOTFLUX_Z', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4', 'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4', 'EBV', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4']


In [4]:
print(fast.colnames)

['TARGETID', 'SURVEY', 'PROGRAM', 'HEALPIX', 'Z', 'COEFF', 'RCHI2', 'RCHI2_CONT', 'RCHI2_PHOT', 'SNR_B', 'SNR_R', 'SNR_Z', 'SMOOTHCORR_B', 'SMOOTHCORR_R', 'SMOOTHCORR_Z', 'VDISP', 'VDISP_IVAR', 'AV', 'AGE', 'ZZSUN', 'LOGMSTAR', 'SFR', 'DN4000', 'DN4000_OBS', 'DN4000_IVAR', 'DN4000_MODEL', 'FLUX_SYNTH_G', 'FLUX_SYNTH_R', 'FLUX_SYNTH_Z', 'FLUX_SYNTH_SPECMODEL_G', 'FLUX_SYNTH_SPECMODEL_R', 'FLUX_SYNTH_SPECMODEL_Z', 'FLUX_SYNTH_PHOTMODEL_G', 'FLUX_SYNTH_PHOTMODEL_R', 'FLUX_SYNTH_PHOTMODEL_Z', 'FLUX_SYNTH_PHOTMODEL_W1', 'FLUX_SYNTH_PHOTMODEL_W2', 'FLUX_SYNTH_PHOTMODEL_W3', 'FLUX_SYNTH_PHOTMODEL_W4', 'KCORR_U', 'ABSMAG_U', 'ABSMAG_IVAR_U', 'KCORR_B', 'ABSMAG_B', 'ABSMAG_IVAR_B', 'KCORR_V', 'ABSMAG_V', 'ABSMAG_IVAR_V', 'KCORR_SDSS_U', 'ABSMAG_SDSS_U', 'ABSMAG_IVAR_SDSS_U', 'KCORR_SDSS_G', 'ABSMAG_SDSS_G', 'ABSMAG_IVAR_SDSS_G', 'KCORR_SDSS_R', 'ABSMAG_SDSS_R', 'ABSMAG_IVAR_SDSS_R', 'KCORR_SDSS_I', 'ABSMAG_SDSS_I', 'ABSMAG_IVAR_SDSS_I', 'KCORR_SDSS_Z', 'ABSMAG_SDSS_Z', 'ABSMAG_IVAR_SDSS_Z', 'KC

#### Select the samples of interest (all from SV3):
1. 5 massive, low-redshift, early-type galaxies.
2. 5 strong [OII]-emitting emission-line galaxies
3. 5 high-redshift QSOs with strong MgII emission
4. 5 low-redshift broad-line AGN

In [5]:
I1 = np.where((meta['SURVEY'] == 'sv3') * (fast['Z'] < 0.4) * (fast['DN4000_MODEL'] > 1.8) * (fast['LOGMSTAR'] > 10.5))[0][:5]

In [6]:
I2 = np.where((meta['SURVEY'] == 'sv3') * (fast['Z'] > 0.7) * 
              (fast['OII_3726_AMP']*np.sqrt(fast['OII_3726_AMP_IVAR']) > 3) *
              (fast['OII_3729_AMP']*np.sqrt(fast['OII_3729_AMP_IVAR']) > 3) *
              ((fast['OII_3726_EW']+fast['OII_3729_EW']) > 50))[0][:5]

In [7]:
I3 = np.where((meta['SURVEY'] == 'sv3') * 
              (fast['MGII_2803_AMP']*np.sqrt(fast['MGII_2803_AMP_IVAR']) > 5) *
              (fast['MGII_2803_SIGMA'] > 1000))[0][:5]

In [8]:
I4 = np.where((meta['SURVEY'] == 'sv3') * (fast['Z'] < 0.3) * 
              (fast['HALPHA_BROAD_SIGMA'] > 1000))[0][:5]

In [9]:
I = np.hstack((I1, I2, I3, I4))
mysample = fast[I]
mysample

TARGETID,SURVEY,PROGRAM,HEALPIX,Z,COEFF [168],RCHI2,RCHI2_CONT,RCHI2_PHOT,SNR_B,SNR_R,SNR_Z,SMOOTHCORR_B,SMOOTHCORR_R,SMOOTHCORR_Z,VDISP,VDISP_IVAR,AV,AGE,ZZSUN,LOGMSTAR,SFR,DN4000,DN4000_OBS,DN4000_IVAR,DN4000_MODEL,FLUX_SYNTH_G,FLUX_SYNTH_R,FLUX_SYNTH_Z,FLUX_SYNTH_SPECMODEL_G,FLUX_SYNTH_SPECMODEL_R,FLUX_SYNTH_SPECMODEL_Z,FLUX_SYNTH_PHOTMODEL_G,FLUX_SYNTH_PHOTMODEL_R,FLUX_SYNTH_PHOTMODEL_Z,FLUX_SYNTH_PHOTMODEL_W1,FLUX_SYNTH_PHOTMODEL_W2,FLUX_SYNTH_PHOTMODEL_W3,FLUX_SYNTH_PHOTMODEL_W4,KCORR_U,ABSMAG_U,ABSMAG_IVAR_U,KCORR_B,ABSMAG_B,ABSMAG_IVAR_B,KCORR_V,ABSMAG_V,ABSMAG_IVAR_V,KCORR_SDSS_U,ABSMAG_SDSS_U,ABSMAG_IVAR_SDSS_U,KCORR_SDSS_G,ABSMAG_SDSS_G,ABSMAG_IVAR_SDSS_G,KCORR_SDSS_R,ABSMAG_SDSS_R,ABSMAG_IVAR_SDSS_R,KCORR_SDSS_I,ABSMAG_SDSS_I,ABSMAG_IVAR_SDSS_I,KCORR_SDSS_Z,ABSMAG_SDSS_Z,ABSMAG_IVAR_SDSS_Z,KCORR_W1,ABSMAG_W1,ABSMAG_IVAR_W1,KCORR_W2,ABSMAG_W2,ABSMAG_IVAR_W2,LOGLNU_1500,LOGLNU_2800,LOGL_5100,FOII_3727_CONT,FHBETA_CONT,FOIII_5007_CONT,FHALPHA_CONT,RCHI2_LINE,DELTA_LINERCHI2,APERCORR,APERCORR_G,APERCORR_R,APERCORR_Z,NARROW_Z,NARROW_ZRMS,BROAD_Z,BROAD_ZRMS,UV_Z,UV_ZRMS,NARROW_SIGMA,NARROW_SIGMARMS,BROAD_SIGMA,BROAD_SIGMARMS,UV_SIGMA,UV_SIGMARMS,MGII_DOUBLET_RATIO,OII_DOUBLET_RATIO,SII_DOUBLET_RATIO,LYALPHA_AMP,LYALPHA_AMP_IVAR,LYALPHA_FLUX,LYALPHA_FLUX_IVAR,LYALPHA_BOXFLUX,LYALPHA_BOXFLUX_IVAR,LYALPHA_VSHIFT,LYALPHA_SIGMA,LYALPHA_CONT,LYALPHA_CONT_IVAR,LYALPHA_EW,LYALPHA_EW_IVAR,LYALPHA_FLUX_LIMIT,LYALPHA_EW_LIMIT,LYALPHA_CHI2,LYALPHA_NPIX,OI_1304_AMP,OI_1304_AMP_IVAR,OI_1304_FLUX,OI_1304_FLUX_IVAR,OI_1304_BOXFLUX,OI_1304_BOXFLUX_IVAR,OI_1304_VSHIFT,OI_1304_SIGMA,OI_1304_CONT,OI_1304_CONT_IVAR,OI_1304_EW,OI_1304_EW_IVAR,OI_1304_FLUX_LIMIT,OI_1304_EW_LIMIT,OI_1304_CHI2,OI_1304_NPIX,SILIV_1396_AMP,SILIV_1396_AMP_IVAR,SILIV_1396_FLUX,SILIV_1396_FLUX_IVAR,SILIV_1396_BOXFLUX,SILIV_1396_BOXFLUX_IVAR,SILIV_1396_VSHIFT,SILIV_1396_SIGMA,SILIV_1396_CONT,SILIV_1396_CONT_IVAR,SILIV_1396_EW,SILIV_1396_EW_IVAR,SILIV_1396_FLUX_LIMIT,SILIV_1396_EW_LIMIT,SILIV_1396_CHI2,SILIV_1396_NPIX,CIV_1549_AMP,CIV_1549_AMP_IVAR,CIV_1549_FLUX,CIV_1549_FLUX_IVAR,CIV_1549_BOXFLUX,CIV_1549_BOXFLUX_IVAR,CIV_1549_VSHIFT,CIV_1549_SIGMA,CIV_1549_CONT,CIV_1549_CONT_IVAR,CIV_1549_EW,CIV_1549_EW_IVAR,CIV_1549_FLUX_LIMIT,CIV_1549_EW_LIMIT,CIV_1549_CHI2,CIV_1549_NPIX,SILIII_1892_AMP,SILIII_1892_AMP_IVAR,SILIII_1892_FLUX,SILIII_1892_FLUX_IVAR,SILIII_1892_BOXFLUX,SILIII_1892_BOXFLUX_IVAR,SILIII_1892_VSHIFT,SILIII_1892_SIGMA,SILIII_1892_CONT,SILIII_1892_CONT_IVAR,SILIII_1892_EW,SILIII_1892_EW_IVAR,SILIII_1892_FLUX_LIMIT,SILIII_1892_EW_LIMIT,SILIII_1892_CHI2,SILIII_1892_NPIX,CIII_1908_AMP,CIII_1908_AMP_IVAR,CIII_1908_FLUX,CIII_1908_FLUX_IVAR,CIII_1908_BOXFLUX,CIII_1908_BOXFLUX_IVAR,CIII_1908_VSHIFT,CIII_1908_SIGMA,CIII_1908_CONT,CIII_1908_CONT_IVAR,CIII_1908_EW,CIII_1908_EW_IVAR,CIII_1908_FLUX_LIMIT,CIII_1908_EW_LIMIT,CIII_1908_CHI2,CIII_1908_NPIX,MGII_2796_AMP,MGII_2796_AMP_IVAR,MGII_2796_FLUX,MGII_2796_FLUX_IVAR,MGII_2796_BOXFLUX,MGII_2796_BOXFLUX_IVAR,MGII_2796_VSHIFT,MGII_2796_SIGMA,MGII_2796_CONT,MGII_2796_CONT_IVAR,MGII_2796_EW,MGII_2796_EW_IVAR,MGII_2796_FLUX_LIMIT,MGII_2796_EW_LIMIT,MGII_2796_CHI2,MGII_2796_NPIX,MGII_2803_AMP,MGII_2803_AMP_IVAR,MGII_2803_FLUX,MGII_2803_FLUX_IVAR,MGII_2803_BOXFLUX,MGII_2803_BOXFLUX_IVAR,MGII_2803_VSHIFT,MGII_2803_SIGMA,MGII_2803_CONT,MGII_2803_CONT_IVAR,MGII_2803_EW,MGII_2803_EW_IVAR,MGII_2803_FLUX_LIMIT,MGII_2803_EW_LIMIT,MGII_2803_CHI2,MGII_2803_NPIX,NEV_3346_AMP,NEV_3346_AMP_IVAR,NEV_3346_FLUX,NEV_3346_FLUX_IVAR,NEV_3346_BOXFLUX,NEV_3346_BOXFLUX_IVAR,NEV_3346_VSHIFT,NEV_3346_SIGMA,NEV_3346_CONT,NEV_3346_CONT_IVAR,NEV_3346_EW,NEV_3346_EW_IVAR,NEV_3346_FLUX_LIMIT,NEV_3346_EW_LIMIT,NEV_3346_CHI2,NEV_3346_NPIX,NEV_3426_AMP,NEV_3426_AMP_IVAR,NEV_3426_FLUX,NEV_3426_FLUX_IVAR,NEV_3426_BOXFLUX,NEV_3426_BOXFLUX_IVAR,NEV_3426_VSHIFT,NEV_3426_SIGMA,NEV_3426_CONT,NEV_3426_CONT_IVAR,NEV_3426_EW,NEV_3426_EW_IVAR,NEV_3426_FLUX_LIMIT,NEV_3426_EW_LIMIT,NEV_3426_CHI2,NEV_3426_NPIX,OII_3726_AMP,OII_3726_AMP_IVAR,OII_3726_FLUX,OII_3726_FLUX_

In [10]:
upload = meta[I]['SURVEY', 'PROGRAM', 'HEALPIX', 'TARGETID']
upload.write(os.path.join(os.environ.get('SCRATCH'), 'upload.fits'), overwrite=True)

In [11]:
upload

SURVEY,PROGRAM,HEALPIX,TARGETID
str7,str6,int32,int64
sv3,dark,11229,1009771901616128
sv3,dark,11253,1009780189560832
sv3,dark,25955,1012939993645056
sv3,dark,11227,1014149739970560
sv3,dark,11230,1014155632967680
sv3,dark,26065,1030610701254657
sv3,bright,9428,1070905056821252
sv3,dark,26282,1083296058441730
sv3,dark,27650,1083332267868160
